<a href="https://colab.research.google.com/github/zooodung/TA-Prediction/blob/main/Encoding_modeling_automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교통사고 위험도 예측

## 라이브러리 및 파일 호출

In [ ]:
!pip install pytimekr

  Preparing metadata (setup.py) ... done
  Created wheel for pytimekr: filename=pytimekr-0.1.0-py3-none-any.whl size=7921 sha256=50929f4c748ef137a78c5de87f85c7c3d5e2d742f7feb455eebfced7fe2f995e
  Stored in directory: /root/.cache/pip/wheels/b3/a3/ba/e73aaae1dec550b8667510c379b019295bba855275258745de
Successfully built pytimekr


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pytimekr import pytimekr
from sklearn.preprocessing import LabelEncoder
import os

In [ ]:
path='/content/drive/MyDrive/camp/project1/open/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
daegu_data=pd.read_csv(path+'train.csv')
wide_data=pd.read_csv(path+'external_open/countrywide_accident.csv')
test_data=pd.read_csv(path+'test.csv')
daegu_data.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


## 기초 통계 분석

In [ ]:
#ta_data_daegu.info()

In [ ]:
#x_test.info()

In [ ]:
#ta_data_daegu.describe()

In [ ]:
#ta_data_daegu.describe()

In [ ]:
#columns = ['요일', '기상상태', '도로형태', '노면상태', '사고유형']

#for column in columns:
#    print(f"Column: {column}")
#    print(ta_data_daegu[column].value_counts())
#    print("-" * 30)

## 전처리

### 대구 데이터 전처리

#### 불필요한 컬럼 제거

In [ ]:
columns_to_drop = ['ID', '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령',
                   '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도']


daegu=daegu_data.drop(columns=columns_to_drop)

#### 사고일시 컬럼 연도, 월, 일, 시간 단위로 분할

In [ ]:
#daegu[['년도','월','일+시간']] = daegu['사고일시'].str.split('-',expand=True)
#daegu[['일','시간']]= daegu['일+시간'].str.split(' ',expand=True)

#daegu.drop(['일+시간','사고일시'],inplace=True, axis=1)
#daegu.drop(['일'],inplace=True, axis=1)

#### 시군구 단위로 분할

In [ ]:
daegu[['시','구','동']] = daegu['시군구'].str.split(' ',expand=True)
daegu.drop(['시군구'],inplace=True, axis=1)

#### 도로 형태 분할

In [ ]:
daegu[['도로형태1', '도로형태2']] = daegu['도로형태'].str.split(' - ', expand=True)
daegu.drop(['도로형태'],inplace=True, axis=1)

In [ ]:
daegu.head()

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타
1,2019-01-01 00,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타
2,2019-01-01 01,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타
3,2019-01-01 02,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타
4,2019-01-01 04,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타


#### 요일 컬럼 평일과 주말로 분류

In [ ]:
#ta_data_daegu['요일 분류'] = ta_data_daegu['요일'].apply(lambda x: '평일' if x in ['월요일', '화요일', '수요일', '목요일', '금요일'] else '주말')

In [ ]:
#ta_data_daegu

### 전국 데이터 전처리

In [ ]:
wide=wide_data.copy()
len(wide)

602775

#### 철길 건널목 제거

In [ ]:
wide = wide[~wide['도로형태'].str.split().str[-1].str.contains('철길건널목')]

#### 불필요한 컬럼 제거

In [ ]:
wide=wide.drop(columns=columns_to_drop)

#### 광역시, 특별시만 분류

In [ ]:
def contains_specific_location(address):
    cities = ['서울특별시', '인천광역시', '부산광역시', '울산광역시', '광주광역시', '대전광역시']
    for city in cities:
        if city in address:
            return True
    return False
wide=wide[wide['시군구'].apply(contains_specific_location)]

wide[['시','구','동']] = wide['시군구'].str.split(' ',expand=True)

In [ ]:
wide['시군구'].apply(lambda x:x.split()[0]).value_counts()

서울특별시    108395
부산광역시     36244
인천광역시     25115
광주광역시     23430
대전광역시     22579
울산광역시     11981
Name: 시군구, dtype: int64

In [ ]:
wide.drop(['시군구'],inplace=True, axis=1)

#### 연도, 월, 일, 시간 분류

In [ ]:
#wide[['년도','월','일+시간']] = wide['사고일시'].str.split('-',expand=True)
#wide[['일','시간']]= wide['일+시간'].str.split(' ',expand=True)

#wide.drop(['일+시간','사고일시'],inplace=True, axis=1)
#wide.drop(['일'],inplace=True, axis=1)

#### 도로 형태 분류

In [ ]:
wide[['도로형태1', '도로형태2']] = wide['도로형태'].str.split(' - ', expand=True)

wide.drop(['도로형태'],inplace=True, axis=1)

In [ ]:
wide.head()

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,강서구,방화동,교차로,교차로횡단보도내
4,2019-01-01 00,화요일,맑음,건조,차대차,0,0,2,1,7,인천광역시,부평구,부평동,교차로,교차로안
5,2019-01-01 00,화요일,맑음,건조,차대사람,0,0,1,0,3,인천광역시,부평구,부평동,교차로,교차로부근
8,2019-01-01 00,화요일,맑음,건조,차대차,0,0,1,0,3,광주광역시,광산구,수완동,단일로,기타
11,2019-01-01 00,화요일,맑음,건조,차대사람,1,0,0,0,10,부산광역시,해운대구,우동,교차로,교차로안


### 데이터 병합 후 전처리 진행

In [ ]:
print(len(daegu),len(wide), len(daegu)+len(wide))

39609 227744 267353


In [ ]:
concatenated_df = pd.concat([daegu, wide])
concatenated_df

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타
1,2019-01-01 00,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타
2,2019-01-01 01,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타
3,2019-01-01 02,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타
4,2019-01-01 04,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,2021-12-31 23,금요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,서초구,반포동,단일로,기타
602769,2021-12-31 23,금요일,맑음,건조,차대차,0,1,0,0,5,부산광역시,연제구,연산동,단일로,기타
602770,2021-12-31 23,금요일,흐림,건조,차대차,0,1,1,0,8,부산광역시,해운대구,우동,교차로,교차로안
602771,2021-12-31 23,금요일,맑음,건조,차대차,0,0,2,0,6,서울특별시,성동구,행당동,단일로,기타


#### 휴일 여부

In [ ]:
for i in range(2019, 2024):
    kr_holidays = pytimekr.holidays(i)
    print(kr_holidays)

[datetime.date(2019, 2, 4), datetime.date(2019, 2, 5), datetime.date(2019, 2, 6), datetime.date(2019, 9, 12), datetime.date(2019, 9, 13), datetime.date(2019, 9, 14), datetime.date(2019, 1, 1), datetime.date(2019, 3, 1), datetime.date(2019, 5, 5), datetime.date(2019, 5, 12), datetime.date(2019, 6, 6), datetime.date(2019, 8, 15), datetime.date(2019, 10, 9), datetime.date(2019, 10, 3), datetime.date(2019, 12, 25)]
[datetime.date(2020, 1, 24), datetime.date(2020, 1, 25), datetime.date(2020, 1, 26), datetime.date(2020, 9, 30), datetime.date(2020, 10, 1), datetime.date(2020, 10, 2), datetime.date(2020, 1, 1), datetime.date(2020, 3, 1), datetime.date(2020, 5, 5), datetime.date(2020, 4, 30), datetime.date(2020, 6, 6), datetime.date(2020, 8, 15), datetime.date(2020, 10, 9), datetime.date(2020, 10, 3), datetime.date(2020, 12, 25)]
[datetime.date(2021, 2, 11), datetime.date(2021, 2, 12), datetime.date(2021, 2, 13), datetime.date(2021, 9, 20), datetime.date(2021, 9, 21), datetime.date(2021, 9, 22)

In [ ]:
kr_holidays = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-05', '2023-05-26', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25']


In [ ]:
concatenated_df['사고일시'] = pd.to_datetime(concatenated_df['사고일시'])

# datetime에서 year, month, day, hour 추출하여 새로운 열 생성
concatenated_df['년도'] = concatenated_df['사고일시'].dt.year
concatenated_df['월'] = concatenated_df['사고일시'].dt.month
concatenated_df['day'] = concatenated_df['사고일시'].dt.day
concatenated_df['시간'] = concatenated_df['사고일시'].dt.hour
concatenated_df['day_of_week'] = concatenated_df['사고일시'].dt.dayofweek

In [ ]:
concatenated_df['holiday'] = np.where((concatenated_df.day_of_week >= 5) | (concatenated_df.사고일시.dt.strftime('%Y-%m-%d').isin(kr_holidays)), '휴일', '평일')

In [ ]:
concatenated_df.drop(columns=['사고일시'], inplace=True)

In [ ]:
concatenated_df['holiday'].value_counts()

평일    195393
휴일     71960
Name: holiday, dtype: int64

In [ ]:
concatenated_df.drop(['day', 'day_of_week'], inplace=True, axis=1)

#### 주말 여부

In [ ]:
#concatenated_df['주말여부']=concatenated_df['요일'].apply(lambda x:'주말' if x in ['토요일','일요일'] else '평일')

#concatenated_df['주말여부'].value_counts()

#### 시간 분류

In [ ]:
def seperate_time(time): #심야, 새벽, 오전, 오후, 저녁, 야간 ->4시간단위
    if 0<=int(time)<4:
        return '심야'
    elif int(time)<8:
        return '새벽'
    elif int(time)<12:
        return '오전'
    elif int(time)<16:
        return '오후'
    elif int(time)<20:
        return '저녁'
    else:
        return '야간'
concatenated_df['시간구분']=concatenated_df['시간'].apply(seperate_time)
print(concatenated_df['시간구분'].value_counts().sum())
concatenated_df['시간구분'].value_counts()

267353


저녁    71768
오후    58743
오전    50058
야간    47737
새벽    20554
심야    18493
Name: 시간구분, dtype: int64

#### 계절 분류

In [ ]:
def seperate_month(month):
    if int(month) in [12,1,2]:
        return '겨울'
    elif int(month) in [3,4,5]:
        return '봄'
    elif int(month) in [6,7,8]:
        return '여름'
    else:
        return '가을'
concatenated_df['계절']=concatenated_df['월'].apply(seperate_month)
print(concatenated_df['계절'].value_counts().sum())
concatenated_df['계절'].value_counts()

267353


가을    70234
여름    68434
봄     66607
겨울    62078
Name: 계절, dtype: int64

### 병합된 데이터 전처리 확인

In [ ]:
concatenated_df

,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타,2019,1,0,휴일,심야,겨울
1,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타,2019,1,0,휴일,심야,겨울
2,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타,2019,1,1,휴일,심야,겨울
3,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타,2019,1,2,휴일,심야,겨울
4,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타,2019,1,4,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,금요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,서초구,반포동,단일로,기타,2021,12,23,평일,야간,겨울
602769,금요일,맑음,건조,차대차,0,1,0,0,5,부산광역시,연제구,연산동,단일로,기타,2021,12,23,평일,야간,겨울
602770,금요일,흐림,건조,차대차,0,1,1,0,8,부산광역시,해운대구,우동,교차로,교차로안,2021,12,23,평일,야간,겨울
602771,금요일,맑음,건조,차대차,0,0,2,0,6,서울특별시,성동구,행당동,단일로,기타,2021,12,23,평일,야간,겨울


### 테스트 데이터 전처리 진행

In [ ]:
test=test_data.copy()

In [ ]:
test[['시','구','동']] = test['시군구'].str.split(' ',expand=True)
test.drop(['시군구'],inplace=True, axis=1)

In [ ]:
test[['도로형태1', '도로형태2']] = test['도로형태'].str.split(' - ', expand=True)
test.drop(['도로형태'],inplace=True, axis=1)

In [ ]:
kr_holidays = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-05', '2023-05-26', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25']


In [ ]:
test['사고일시'] = pd.to_datetime(test['사고일시'])

# datetime에서 year, month, day, hour 추출하여 새로운 열 생성
test['년도'] = test['사고일시'].dt.year
test['월'] = test['사고일시'].dt.month
test['day'] = test['사고일시'].dt.day
test['시간'] = test['사고일시'].dt.hour
test['day_of_week'] = test['사고일시'].dt.dayofweek

test['holiday'] = np.where((test.day_of_week >= 5) | (test.사고일시.dt.strftime('%Y-%m-%d').isin(kr_holidays)), '휴일', '평일')

test.drop(columns=['사고일시'], inplace=True)
test.drop(['day', 'day_of_week'], inplace=True, axis=1)

In [ ]:
test['시간구분']=test['시간'].apply(seperate_time)

In [ ]:
test['계절']=test['월'].apply(seperate_month)
test['계절'].value_counts()

가을    2947
여름    2783
봄     2771
겨울    2462
Name: 계절, dtype: int64

In [ ]:
test.drop('ID',axis=1,inplace=True)

#### 테스트 데이터 전처리 결과 확인

In [ ]:
test

,요일,기상상태,노면상태,사고유형,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,토요일,맑음,건조,차대사람,대구광역시,수성구,상동,교차로,교차로안,2022,1,1,휴일,심야,겨울
1,토요일,맑음,건조,차대사람,대구광역시,수성구,지산동,단일로,기타,2022,1,1,휴일,심야,겨울
2,토요일,맑음,건조,차대차,대구광역시,수성구,수성동2가,교차로,교차로안,2022,1,4,휴일,새벽,겨울
3,토요일,맑음,건조,차대차,대구광역시,수성구,신매동,단일로,기타,2022,1,4,휴일,새벽,겨울
4,토요일,맑음,건조,차대차,대구광역시,달서구,감삼동,교차로,교차로안,2022,1,6,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,토요일,맑음,건조,차대차,대구광역시,남구,대명동,단일로,터널,2022,12,18,휴일,저녁,겨울
10959,토요일,맑음,건조,차대차,대구광역시,수성구,시지동,단일로,기타,2022,12,18,휴일,저녁,겨울
10960,토요일,맑음,건조,차대차,대구광역시,수성구,연호동,단일로,기타,2022,12,20,휴일,야간,겨울
10961,토요일,맑음,건조,차대차,대구광역시,수성구,범물동,교차로,교차로부근,2022,12,20,휴일,야간,겨울


#### 합쳐진 훈련 데이터 셋 오류 수정

In [ ]:
concatenated_df = concatenated_df[~concatenated_df['기상상태'].str.split().str[-1].str.contains('안개')]
concatenated_df = concatenated_df[~concatenated_df['노면상태'].str.split().str[-1].str.contains('해빙')]

In [ ]:
concatenated_df

,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타,2019,1,0,휴일,심야,겨울
1,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타,2019,1,0,휴일,심야,겨울
2,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타,2019,1,1,휴일,심야,겨울
3,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타,2019,1,2,휴일,심야,겨울
4,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타,2019,1,4,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,금요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,서초구,반포동,단일로,기타,2021,12,23,평일,야간,겨울
602769,금요일,맑음,건조,차대차,0,1,0,0,5,부산광역시,연제구,연산동,단일로,기타,2021,12,23,평일,야간,겨울
602770,금요일,흐림,건조,차대차,0,1,1,0,8,부산광역시,해운대구,우동,교차로,교차로안,2021,12,23,평일,야간,겨울
602771,금요일,맑음,건조,차대차,0,0,2,0,6,서울특별시,성동구,행당동,단일로,기타,2021,12,23,평일,야간,겨울


#### 전처리 데이터 저장 - 전처리 매번 로딩할 필요 없다

In [ ]:
# concatenated_df.to_csv(path + f'/데이터들/인코딩 없음/train.csv')
# test.to_csv(path + f'/데이터들/인코딩 없음/test.csv')

In [ ]:
# path = os.getcwd()

# concatenated_df.to_csv(path + f'/데이터들/인코딩 없음/train.csv')
# test = pd.read_csv(path + f'/데이터들/인코딩 없음/test.csv')

#전처리 - 원핫인코딩

In [ ]:
total_data = pd.concat([concatenated_df, test])
total_data = pd.get_dummies(total_data)

In [ ]:
for c in total_data.columns:
    print(c)
print(len(total_data.columns))

사망자수
중상자수
경상자수
부상자수
ECLO
년도
월
시간
요일_금요일
요일_목요일
요일_수요일
요일_월요일
요일_일요일
요일_토요일
요일_화요일
기상상태_기타
기상상태_눈
기상상태_맑음
기상상태_비
기상상태_흐림
노면상태_건조
노면상태_기타
노면상태_서리/결빙
노면상태_적설
노면상태_젖음/습기
노면상태_침수
사고유형_차대사람
사고유형_차대차
사고유형_차량단독
시_광주광역시
시_대구광역시
시_대전광역시
시_부산광역시
시_서울특별시
시_울산광역시
시_인천광역시
구_강남구
구_강동구
구_강북구
구_강서구
구_강화군
구_계양구
구_관악구
구_광산구
구_광진구
구_구로구
구_금정구
구_금천구
구_기장군
구_남구
구_남동구
구_노원구
구_달서구
구_달성군
구_대덕구
구_도봉구
구_동구
구_동대문구
구_동래구
구_동작구
구_마포구
구_미추홀구
구_부산진구
구_부평구
구_북구
구_사상구
구_사하구
구_서구
구_서대문구
구_서초구
구_성동구
구_성북구
구_송파구
구_수성구
구_수영구
구_양천구
구_연수구
구_연제구
구_영도구
구_영등포구
구_옹진군
구_용산구
구_울주군
구_유성구
구_은평구
구_종로구
구_중구
구_중랑구
구_해운대구
동_가대동
동_가락동
동_가리봉동
동_가산동
동_가수원동
동_가야동
동_가양동
동_가오동
동_가장동
동_가정동
동_가좌동
동_가창면
동_가천동
동_가회동
동_각산동
동_각화동
동_간석동
동_갈마동
동_갈산동
동_갈월동
동_갈전동
동_갈현동
동_감만동
동_감삼동
동_감전동
동_감천동
동_갑동
동_강동동
동_강일동
동_강화읍
동_개금동
동_개봉동
동_개포동
동_개화동
동_거여동
동_거제동
동_검단동
동_검사동
동_검암동
동_견지동
동_경동
동_경서동
동_경운동
동_계동
동_계림동
동_계산동
동_계산동1가
동_계산동2가
동_고덕동
동_고룡동
동_고모동
동_고사동
동_고성동1가
동_고성동2가
동_고성동3가
동_고잔동
동_고척동
동_공덕동
동_공릉동
동_공촌동
동_공평동
동_공항동
동_과해동
동_관교동
동_관동1가
동_관동2가
동_관동3가
동_관수동
동_

In [ ]:
train_data = total_data[~total_data['ECLO'].isnull()]
test_data = total_data[total_data['ECLO'].isnull()]

In [ ]:
test_data = test_data.drop(['사망자수','중상자수','경상자수','부상자수','ECLO'], axis=1)

In [ ]:
train_data

,사망자수,중상자수,경상자수,부상자수,ECLO,년도,월,시간,요일_금요일,요일_목요일,...,시간구분_새벽,시간구분_심야,시간구분_야간,시간구분_오전,시간구분_오후,시간구분_저녁,계절_가을,계절_겨울,계절_봄,계절_여름
0,0.0,1.0,0.0,0.0,5.0,2019,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,0.0,0.0,1.0,0.0,3.0,2019,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,0.0,0.0,1.0,0.0,3.0,2019,1,1,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0.0,1.0,0.0,0.0,5.0,2019,1,2,0,0,...,0,1,0,0,0,0,0,1,0,0
4,0.0,0.0,1.0,0.0,3.0,2019,1,4,0,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,0.0,0.0,1.0,0.0,3.0,2021,12,23,1,0,...,0,0,1,0,0,0,0,1,0,0
602769,0.0,1.0,0.0,0.0,5.0,2021,12,23,1,0,...,0,0,1,0,0,0,0,1,0,0
602770,0.0,1.0,1.0,0.0,8.0,2021,12,23,1,0,...,0,0,1,0,0,0,0,1,0,0
602771,0.0,0.0,2.0,0.0,6.0,2021,12,23,1,0,...,0,0,1,0,0,0,0,1,0,0


In [ ]:
test_data

,년도,월,시간,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,...,시간구분_새벽,시간구분_심야,시간구분_야간,시간구분_오전,시간구분_오후,시간구분_저녁,계절_가을,계절_겨울,계절_봄,계절_여름
0,2022,1,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
1,2022,1,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2,2022,1,4,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
3,2022,1,4,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,2022,1,6,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,2022,12,18,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
10959,2022,12,18,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
10960,2022,12,20,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
10961,2022,12,20,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267300 entries, 0 to 602772
Columns: 1399 entries, 사망자수 to 계절_여름
dtypes: float64(5), int64(3), uint8(1391)
memory usage: 372.9 MB


#AutoML을 이용한 하이퍼파라마터 튜닝

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.9 MB/s eta 0:00:00


In [ ]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error as msle


label값 분리

In [ ]:
y1=train_data[['ECLO']]
y2=train_data[['사망자수']]
y3=train_data[['중상자수']]
y4=train_data[['경상자수']]
y5=train_data[['부상자수']]
X=train_data.drop(columns=['사망자수','중상자수','경상자수','부상자수','ECLO'])

훈련,테스트(검증)셋 분리

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y1, test_size=0.2, random_state=42)

XGBRegressor

In [ ]:
def objective(trial: Trial):
    params_xgb={
        'random_state':42,
        'max_depth':trial.suggest_int('max_depth',3,10),                        #트리의 최대 깊이 3~10
        'learning_rate':trial.suggest_float('learning_rate',0.0001,0.1),        #학습률 0.0001~0.1
        'n_estimators':trial.suggest_int('n_estimators',100,1000,100),          #트리 개수 100~1000 100단위
        'reg_alpha':trial.suggest_float('reg_alpha',0.01,1.0),                  #L1규제
        'reg_lambda':trial.suggest_float('reg_lambda',0.01,1.0),                #L2규제
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),                #학습 시 사용할 훈련 데이터의 비율
        'eta': trial.suggest_float('eta', 0.007, 0.013),                        #부스팅 각 단계에서 사용되는 축소 비율
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),  #트리 구성하는데 사용할 특성의 비율
    }
    model=XGBRegressor(**params_xgb,n_jobs=-1)
    best_xgb=model.fit(X_train,y_train, eval_set=[(X_test,y_test)],eval_metric='rmsle',early_stopping_rounds=100)

    cgb_pred = model.predict(X_test)
    loss=msle(y_test,cgb_pred)

    return np.sqrt(loss)

In [ ]:
sampler = TPESampler(seed=42)
study1 = optuna.create_study(
    study_name="xgb_parameter_opt",
    direction="minimize",
    sampler=sampler,
)

study1.optimize(objective, n_trials=10)
best_xgb_model=XGBRegressor(**study1.best_trial.params,n_jobs=-1)
best_xgb_model.fit(X_train,y_train, eval_set=[(X_test,y_test)], early_stopping_rounds=100, eval_metric='rmsle',verbose=False)
print("Best Score:", study1.best_value)
print("Best trial:", study1.best_trial.params)

[I 2024-03-05 08:49:49,220] A new study created in memory with name: xgb_parameter_opt
<ipython-input-64-25e3329bee13>:6: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_estimators':trial.suggest_int('n_estimators',100,1000,100),          #트리 개수 100~1000 100단위
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmsle:0.44127
[1]	validation_0-rmsle:0.44024
[2]	validation_0-rmsle:0.43945
[3]	validation_0-rmsle:0.43872
[4]	validation_0-rmsle:0.43816
[5]	validation_0-rmsle:0.43766
[6]	validation_0-rmsle:0.43727
[7]	validation_0-rmsle:0.43692
[8]	validation_0-rmsle:0.43669
[9]	validation_0-rmsle:0.43645
[10]	validation_0-rmsle:0.43630
[11]	validation_0-rmsle:0.43617
[12]	validation_0-rmsle:0.43603
[13]	validation_0-rmsle:0.43591
[14]	validation_0-rmsle:0.43581
[15]	validation_0-rmsle:0.43566
[16]	validation_0-rmsle:0.43563
[17]	validation_0-rmsle:0.43553
[18]	validation_0-rmsle:0.43546
[19]	validation_0-rmsle:0.43538
[20]	validation_0-rmsle:0.43532
[21]	validation_0-rmsle:0.43529
[22]	validation_0-rmsle:0.43524
[23]	validation_0-rmsle:0.43519
[24]	validation_0-rmsle:0.43518
[25]	validation_0-rmsle:0.43517
[26]	validation_0-rmsle:0.43510
[27]	validation_0-rmsle:0.43505
[28]	validation_0-rmsle:0.43504
[29]	validation_0-rmsle:0.43506
[30]	validation_0-rmsle:0.43505
[31]	validation_0-

[I 2024-03-05 08:57:05,570] Trial 0 finished with value: 0.4344659675987801 and parameters: {'max_depth': 5, 'learning_rate': 0.09507635921035062, 'n_estimators': 800, 'reg_alpha': 0.6026718993550663, 'reg_lambda': 0.16445845403801215, 'subsample': 0.7467983561008608, 'eta': 0.007348501673009197, 'colsample_bytree': 0.9197056874649611}. Best is trial 0 with value: 0.4344659675987801.
<ipython-input-64-25e3329bee13>:6: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_estimators':trial.suggest_int('n_estimators',100,1000,100),          #트리 개수 100~1000 100단위
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is 

[0]	validation_0-rmsle:0.44158
[1]	validation_0-rmsle:0.44116
[2]	validation_0-rmsle:0.44043
[3]	validation_0-rmsle:0.43972
[4]	validation_0-rmsle:0.43917
[5]	validation_0-rmsle:0.43858
[6]	validation_0-rmsle:0.43814
[7]	validation_0-rmsle:0.43774
[8]	validation_0-rmsle:0.43740
[9]	validation_0-rmsle:0.43709
[10]	validation_0-rmsle:0.43682
[11]	validation_0-rmsle:0.43655
[12]	validation_0-rmsle:0.43636
[13]	validation_0-rmsle:0.43614
[14]	validation_0-rmsle:0.43599
[15]	validation_0-rmsle:0.43580
[16]	validation_0-rmsle:0.43567
[17]	validation_0-rmsle:0.43557
[18]	validation_0-rmsle:0.43549
[19]	validation_0-rmsle:0.43538
[20]	validation_0-rmsle:0.43531
[21]	validation_0-rmsle:0.43526
[22]	validation_0-rmsle:0.43520
[23]	validation_0-rmsle:0.43515
[24]	validation_0-rmsle:0.43509
[25]	validation_0-rmsle:0.43506
[26]	validation_0-rmsle:0.43502
[27]	validation_0-rmsle:0.43499
[28]	validation_0-rmsle:0.43496
[29]	validation_0-rmsle:0.43493
[30]	validation_0-rmsle:0.43492
[31]	validation_0-

[I 2024-03-05 09:00:34,224] Trial 1 finished with value: 0.4341972597454927 and parameters: {'max_depth': 7, 'learning_rate': 0.07083645052182495, 'n_estimators': 100, 'reg_alpha': 0.9702107536403743, 'reg_lambda': 0.8341182143924175, 'subsample': 0.7637017332034828, 'eta': 0.008090949803242604, 'colsample_bytree': 0.5100427059120604}. Best is trial 1 with value: 0.4341972597454927.
<ipython-input-64-25e3329bee13>:6: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_estimators':trial.suggest_int('n_estimators',100,1000,100),          #트리 개수 100~1000 100단위
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is d

...

[0]	validation_0-rmsle:0.44188
[1]	validation_0-rmsle:0.44158
[2]	validation_0-rmsle:0.44101
[3]	validation_0-rmsle:0.44044
[4]	validation_0-rmsle:0.44000
[5]	validation_0-rmsle:0.43953
[6]	validation_0-rmsle:0.43915
[7]	validation_0-rmsle:0.43884
[8]	validation_0-rmsle:0.43851
[9]	validation_0-rmsle:0.43824
[10]	validation_0-rmsle:0.43797
[11]	validation_0-rmsle:0.43772
[12]	validation_0-rmsle:0.43753
[13]	validation_0-rmsle:0.43728
[14]	validation_0-rmsle:0.43710
[15]	validation_0-rmsle:0.43691
[16]	validation_0-rmsle:0.43676
[17]	validation_0-rmsle:0.43659
[18]	validation_0-rmsle:0.43646
[19]	validation_0-rmsle:0.43634
[20]	validation_0-rmsle:0.43625
[21]	validation_0-rmsle:0.43617
[22]	validation_0-rmsle:0.43607
[23]	validation_0-rmsle:0.43598
[24]	validation_0-rmsle:0.43590
[25]	validation_0-rmsle:0.43582
[26]	validation_0-rmsle:0.43576
[27]	validation_0-rmsle:0.43569
[28]	validation_0-rmsle:0.43565
[29]	validation_0-rmsle:0.43560
[30]	validation_0-rmsle:0.43553
[31]	validation_0-

[I 2024-03-05 10:00:33,164] Trial 8 finished with value: 0.43416789955647334 and parameters: {'max_depth': 5, 'learning_rate': 0.05431533870750903, 'n_estimators': 200, 'reg_alpha': 0.8041750109464992, 'reg_lambda': 0.08380513724297312, 'subsample': 0.9960660809801551, 'eta': 0.011633468615779945, 'colsample_bytree': 0.5192294089205034}. Best is trial 4 with value: 0.4341566370510967.
<ipython-input-64-25e3329bee13>:6: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_estimators':trial.suggest_int('n_estimators',100,1000,100),          #트리 개수 100~1000 100단위
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is

[0]	validation_0-rmsle:0.44174
[1]	validation_0-rmsle:0.44141
[2]	validation_0-rmsle:0.44084
[3]	validation_0-rmsle:0.44011
[4]	validation_0-rmsle:0.43966
[5]	validation_0-rmsle:0.43947
[6]	validation_0-rmsle:0.43904
[7]	validation_0-rmsle:0.43868
[8]	validation_0-rmsle:0.43832
[9]	validation_0-rmsle:0.43802
[10]	validation_0-rmsle:0.43775
[11]	validation_0-rmsle:0.43756
[12]	validation_0-rmsle:0.43741
[13]	validation_0-rmsle:0.43723
[14]	validation_0-rmsle:0.43714
[15]	validation_0-rmsle:0.43696
[16]	validation_0-rmsle:0.43690
[17]	validation_0-rmsle:0.43681
[18]	validation_0-rmsle:0.43677
[19]	validation_0-rmsle:0.43670
[20]	validation_0-rmsle:0.43666
[21]	validation_0-rmsle:0.43660
[22]	validation_0-rmsle:0.43653
[23]	validation_0-rmsle:0.43646
[24]	validation_0-rmsle:0.43642
[25]	validation_0-rmsle:0.43635
[26]	validation_0-rmsle:0.43629
[27]	validation_0-rmsle:0.43627
[28]	validation_0-rmsle:0.43623
[29]	validation_0-rmsle:0.43622
[30]	validation_0-rmsle:0.43619
[31]	validation_0-

[I 2024-03-05 10:13:30,564] Trial 9 finished with value: 0.43433863985023297 and parameters: {'max_depth': 3, 'learning_rate': 0.08156459670263794, 'n_estimators': 800, 'reg_alpha': 0.7317170963605775, 'reg_lambda': 0.7735576432190863, 'subsample': 0.7222133955202271, 'eta': 0.009150794371265637, 'colsample_bytree': 0.4695214357150779}. Best is trial 4 with value: 0.4341566370510967.
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Best Score: 0.4341566370510967
Best trial: {'max_depth': 3, 'learning_rate': 0.094893665171608, 'n_estimators': 1000, 'reg_alpha': 0.8103133746352965, 'reg_lambda': 0.31156763148163696, 'subsample': 0.7293016342019151, 'eta': 0.011105398159072941, 'colsample_bytree': 0.6640914962437607}


In [ ]:
import joblib

# XGBoost 모델 저장
joblib.dump(best_xgb_model, 'best_xgb_model.pkl')


['best_xgb_model.pkl']

In [ ]:
optuna.visualization.plot_optimization_history(study1)

In [ ]:
#파라미터들관의 관계
optuna.visualization.plot_parallel_coordinate(study1)

lightlgbm

In [ ]:
from lightgbm import LGBMRegressor, early_stopping

In [ ]:
def objective2(trial:Trial):
    params_lgb ={
        'random_state':42,
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True),   #최대 잎의 수
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),     #학습률
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000,100),           #트리 개수
        'subsample': trial.suggest_loguniform('subsample', 0.4,1),                  #학습 시 사용할 훈련 데이터의 비율
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),      #트리를 구성하는 데 사용할 특성의 비율
        'reg_alpha':trial.suggest_float('reg_alpha',0.01,1.0),                      #L1규제
        'reg_lambda':trial.suggest_float('reg_lambda',0.01,1.0),                    #L2규제
        'max_depth': trial.suggest_int('max_depth',3, 10),                          #최대 깊이
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),        #리프 노드에서 필요핸 최소 샘플 개수
    }
    model=LGBMRegressor(**params_lgb,n_jobs=-1)
    best_lgbm=model.fit(X_train,y_train, eval_set=[(X_test,y_test)],eval_metric='rmsle', callbacks=[early_stopping(stopping_rounds=100)])

    lgbm_pred = model.predict(X_test)
    loss=msle(y_test,lgbm_pred)

    return np.sqrt(loss)


In [ ]:
sampler = TPESampler(seed=42)
study2 = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)

study2.optimize(objective2, n_trials=10)
best_lgbm_model = LGBMRegressor(**study2.best_trial.params,n_jobs=-1)
best_lgbm_model.fit(X_train,y_train, eval_set=[(X_test,y_test)],eval_metric='rmsle', callbacks=[early_stopping(stopping_rounds=100)])
print("Best Score:", study2.best_value)
print("Best trial:", study2.best_trial.params)

[I 2024-03-05 10:23:57,901] A new study created in memory with name: lgbm_parameter_opt
<ipython-input-70-22fdc0749213>:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

<ipython-input-70-22fdc0749213>:8: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

<ipython-input-70-22fdc0749213>:9: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.121995 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1555
[LightGBM] [Info] Number of data points in the train set: 213840, number of used features: 760
[LightGBM] [Info] Start training from score 4.632730
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[800]	valid_0's rmse: 3.00756


[I 2024-03-05 10:24:30,805] Trial 0 finished with value: 0.43427763071455167 and parameters: {'num_leaves': 17, 'learning_rate': 0.005061576888752304, 'n_estimators': 800, 'subsample': 0.6922933401417407, 'colsample_bytree': 0.4936111842654619, 'reg_alpha': 0.16443457513284063, 'reg_lambda': 0.06750277604651747, 'max_depth': 9, 'min_child_samples': 62}. Best is trial 0 with value: 0.43427763071455167.
<ipython-input-70-22fdc0749213>:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

<ipython-input-70-22fdc0749213>:8: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

<ipython-input-70-22fdc0749213>:9: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030861 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1599
[LightGBM] [Info] Number of data points in the train set: 213840, number of used features: 782
[LightGBM] [Info] Start training from score 4.632730
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

[I 2024-03-05 10:25:09,740] Trial 1 finished with value: 0.44261166442696137 and parameters: {'num_leaves': 152, 'learning_rate': 1.328944872286916e-08, 'n_estimators': 1000, 'subsample': 0.8576739569300319, 'colsample_bytree': 0.5274034664069657, 'reg_alpha': 0.19000671753502962, 'reg_lambda': 0.1915704647548995, 'max_depth': 5, 'min_child_samples': 55}. Best is trial 0 with value: 0.43427763071455167.
<ipython-input-70-22fdc0749213>:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

<ipython-input-70-22fdc0749213>:8: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

<ipython-input-70-22fdc0749213>:9: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/

...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030406 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1349
[LightGBM] [Info] Number of data points in the train set: 213840, number of used features: 657
[LightGBM] [Info] Start training from score 4.632730
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[800]	valid_0's rmse: 3.04587


[I 2024-03-05 10:27:44,146] Trial 8 finished with value: 0.4397605430350371 and parameters: {'num_leaves': 2, 'learning_rate': 0.0007812113973534924, 'n_estimators': 800, 'subsample': 0.7801194575432674, 'colsample_bytree': 0.8627622080115674, 'reg_alpha': 0.08330420521674946, 'reg_lambda': 0.3648810712588299, 'max_depth': 3, 'min_child_samples': 87}. Best is trial 0 with value: 0.43427763071455167.
<ipython-input-70-22fdc0749213>:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

<ipython-input-70-22fdc0749213>:8: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

<ipython-input-70-22fdc0749213>:9: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029891 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1655
[LightGBM] [Info] Number of data points in the train set: 213840, number of used features: 810
[LightGBM] [Info] Start training from score 4.632730
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 3.06383


[I 2024-03-05 10:27:56,184] Trial 9 finished with value: 0.4426104454858959 and parameters: {'num_leaves': 88, 'learning_rate': 9.669146694016007e-07, 'n_estimators': 100, 'subsample': 0.5318783332418758, 'colsample_bytree': 0.5951099932160482, 'reg_alpha': 0.7323101165546835, 'reg_lambda': 0.641181896641661, 'max_depth': 10, 'min_child_samples': 50}. Best is trial 0 with value: 0.43427763071455167.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032897 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1549
[LightGBM] [Info] Number of data points in the train set: 213840, number of used features: 757
[LightGBM] [Info] Start training from score 4.632730
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[800]	valid_0's l2: 9.04528
Best Score: 0.43427763071455167
Best trial: {'num_leaves': 17, 'learning_rate': 0.005061576888752304, 'n_estimators': 800, 'subsample': 0.6922933401417407, 'colsample_bytree': 0.4936111842654619, 'reg_alpha': 0.16443457513284063, 'reg_lambda': 0.06750277604651747, 'max_depth': 9, 'min_child_samples': 62}


In [ ]:
joblib.dump(best_lgbm_model, 'best_lgbm_model.pkl')

['best_lgbm_model.pkl']

catboost

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 4.2 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
def objective3(trial:Trial):
    params_cat ={
        'iterations':trial.suggest_int("iterations", 100, 1000,100),                            #부스팅 반복 횟수
        'od_wait':trial.suggest_int('od_wait', 500, 2300),                                      #조기 정지를 위한 반복 횟수
        'learning_rate' : trial.suggest_uniform('learning_rate',1e-8, 1),                       #학습률
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),                             #L2규제
        'subsample': trial.suggest_uniform('subsample',0.4,1),                                  #각 부스팅 단계에서 사용되는 훈련 데이터의 샘플링 비율
        'random_strength': trial.suggest_uniform('random_strength',10,50),                      #데이터 샘플의 무작위성을 조절
        'depth': trial.suggest_int('depth',1, 15),                                              #트리의 최대 깊이
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),                         #리프 노드에 필요한 최소 데이터 수
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),     #리프 노드의 추정 반복 횟수
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),   #부트스트랩 샘플링 온도
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),                 #각 트리 수준에서 특성을 샘플링하는 비율
    }
    model=CatBoostRegressor(**params_cat)
    best_cat=model.fit(X_train, y_train, eval_set=[(X_test,y_test)],early_stopping_rounds=100)

    cat_pred = model.predict(X_test)
    loss=msle(y_test,cat_pred)

    return np.sqrt(loss)


In [ ]:
sampler = TPESampler(seed=42)
study3 = optuna.create_study(
    study_name="cat_parameter_opt",
    direction="minimize",
    sampler=sampler,
)

study3.optimize(objective3, n_trials=10)
best_cat_model=CatBoostRegressor(**study3.best_trial.params)
best_cat_model.fit(X_train,y_train, eval_set=[(X_test,y_test)], early_stopping_rounds=100,verbose=False)
print("Best Score:", study3.best_value)
print("Best trial:", study3.best_trial.params)

[I 2024-03-05 10:28:41,039] A new study created in memory with name: cat_parameter_opt
<ipython-input-75-8e3e5ad8c07c>:3: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

<ipython-input-75-8e3e5ad8c07c>:5: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

<ipython-input-75-8e3e5ad8c07c>:6: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

<ipython-input-75-8e3e5ad8c07c>:7: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

<ipython-input-75-8e3e5ad8c07c>:8: FutureWarning:

suggest_uniform has been depre

0:	learn: 3.1941273	test: 3.0638830	best: 3.0638830 (0)	total: 163ms	remaining: 1m 5s
1:	learn: 3.1910938	test: 3.0602882	best: 3.0602882 (1)	total: 250ms	remaining: 49.7s
2:	learn: 3.1907747	test: 3.0601287	best: 3.0601287 (2)	total: 300ms	remaining: 39.7s
3:	learn: 3.1718068	test: 3.0406142	best: 3.0406142 (3)	total: 350ms	remaining: 34.7s
4:	learn: 3.1715726	test: 3.0402740	best: 3.0402740 (4)	total: 419ms	remaining: 33.1s
5:	learn: 3.1696364	test: 3.0384164	best: 3.0384164 (5)	total: 470ms	remaining: 30.8s
6:	learn: 3.1695703	test: 3.0383081	best: 3.0383081 (6)	total: 537ms	remaining: 30.2s
7:	learn: 3.1695489	test: 3.0382835	best: 3.0382835 (7)	total: 602ms	remaining: 29.5s
8:	learn: 3.1695445	test: 3.0382553	best: 3.0382553 (8)	total: 661ms	remaining: 28.7s
9:	learn: 3.1679726	test: 3.0363976	best: 3.0363976 (9)	total: 731ms	remaining: 28.5s
10:	learn: 3.1636122	test: 3.0318442	best: 3.0318442 (10)	total: 779ms	remaining: 27.6s
11:	learn: 3.1632988	test: 3.0315258	best: 3.0315258

[I 2024-03-05 10:29:09,304] Trial 0 finished with value: 0.4357812453468696 and parameters: {'iterations': 400, 'od_wait': 2212, 'learning_rate': 0.7319939444914657, 'reg_lambda': 59.86585243311882, 'subsample': 0.4936111842654619, 'random_strength': 16.239780813448107, 'depth': 1, 'min_data_in_leaf': 26, 'leaf_estimation_iterations': 10, 'bagging_temperature': 6.79657809075816, 'colsample_bylevel': 0.41235069657748147}. Best is trial 0 with value: 0.4357812453468696.
<ipython-input-75-8e3e5ad8c07c>:3: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

<ipython-input-75-8e3e5ad8c07c>:5: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

<ipython-input-75-8e3e5ad8c07c>:6: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See h

0:	learn: 3.1820236	test: 3.0515023	best: 3.0515023 (0)	total: 56.8ms	remaining: 56.7s
1:	learn: 3.1758064	test: 3.0448603	best: 3.0448603 (1)	total: 93.3ms	remaining: 46.5s
2:	learn: 3.1718014	test: 3.0406767	best: 3.0406767 (2)	total: 139ms	remaining: 46.3s
3:	learn: 3.1682939	test: 3.0370092	best: 3.0370092 (3)	total: 180ms	remaining: 44.7s
4:	learn: 3.1661019	test: 3.0349060	best: 3.0349060 (4)	total: 220ms	remaining: 43.8s
5:	learn: 3.1658512	test: 3.0346429	best: 3.0346429 (5)	total: 249ms	remaining: 41.2s
6:	learn: 3.1635880	test: 3.0322218	best: 3.0322218 (6)	total: 303ms	remaining: 42.9s
7:	learn: 3.1601526	test: 3.0289788	best: 3.0289788 (7)	total: 351ms	remaining: 43.5s
8:	learn: 3.1590111	test: 3.0278444	best: 3.0278444 (8)	total: 397ms	remaining: 43.7s
9:	learn: 3.1573710	test: 3.0264145	best: 3.0264145 (9)	total: 445ms	remaining: 44s
10:	learn: 3.1571279	test: 3.0261218	best: 3.0261218 (10)	total: 478ms	remaining: 43s
11:	learn: 3.1567884	test: 3.0258552	best: 3.0258552 (

[I 2024-03-05 10:29:32,539] Trial 1 finished with value: 0.4346464357752033 and parameters: {'iterations': 1000, 'od_wait': 1999, 'learning_rate': 0.21233911855488502, 'reg_lambda': 18.182504902460387, 'subsample': 0.5100427059120604, 'random_strength': 22.16968971838151, 'depth': 8, 'min_data_in_leaf': 13, 'leaf_estimation_iterations': 5, 'bagging_temperature': 2.801635158716261, 'colsample_bylevel': 0.4836963163912251}. Best is trial 1 with value: 0.4346464357752033.
<ipython-input-75-8e3e5ad8c07c>:3: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

<ipython-input-75-8e3e5ad8c07c>:5: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

<ipython-input-75-8e3e5ad8c07c>:6: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See 

...

0:	learn: 3.1882650	test: 3.0577908	best: 3.0577908 (0)	total: 68.2ms	remaining: 1m 1s
1:	learn: 3.1846217	test: 3.0541924	best: 3.0541924 (1)	total: 355ms	remaining: 2m 39s
2:	learn: 3.1789837	test: 3.0486094	best: 3.0486094 (2)	total: 659ms	remaining: 3m 17s
3:	learn: 3.1749960	test: 3.0447916	best: 3.0447916 (3)	total: 953ms	remaining: 3m 33s
4:	learn: 3.1741616	test: 3.0440329	best: 3.0440329 (4)	total: 1.04s	remaining: 3m 6s
5:	learn: 3.1697186	test: 3.0393409	best: 3.0393409 (5)	total: 1.33s	remaining: 3m 18s
6:	learn: 3.1665439	test: 3.0361086	best: 3.0361086 (6)	total: 1.63s	remaining: 3m 28s
7:	learn: 3.1642723	test: 3.0339304	best: 3.0339304 (7)	total: 1.92s	remaining: 3m 34s
8:	learn: 3.1628596	test: 3.0325804	best: 3.0325804 (8)	total: 2.03s	remaining: 3m 21s
9:	learn: 3.1616647	test: 3.0316852	best: 3.0316852 (9)	total: 2.33s	remaining: 3m 26s
10:	learn: 3.1585943	test: 3.0286231	best: 3.0286231 (10)	total: 2.63s	remaining: 3m 32s
11:	learn: 3.1581492	test: 3.0285024	best:

[I 2024-03-05 10:36:07,268] Trial 8 finished with value: 0.434446291562222 and parameters: {'iterations': 900, 'od_wait': 1350, 'learning_rate': 0.11959425474235923, 'reg_lambda': 71.32448158985163, 'subsample': 0.8564710291701385, 'random_strength': 32.45108790277985, 'depth': 12, 'min_data_in_leaf': 15, 'leaf_estimation_iterations': 8, 'bagging_temperature': 0.5130551760589835, 'colsample_bylevel': 0.4152514760464571}. Best is trial 8 with value: 0.434446291562222.
<ipython-input-75-8e3e5ad8c07c>:3: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

<ipython-input-75-8e3e5ad8c07c>:5: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

<ipython-input-75-8e3e5ad8c07c>:6: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See ht

0:	learn: 3.1896193	test: 3.0589377	best: 3.0589377 (0)	total: 48.6ms	remaining: 9.68s
1:	learn: 3.1891467	test: 3.0582950	best: 3.0582950 (1)	total: 95.4ms	remaining: 9.44s
2:	learn: 3.1856585	test: 3.0543725	best: 3.0543725 (2)	total: 140ms	remaining: 9.22s
3:	learn: 3.1648053	test: 3.0332842	best: 3.0332842 (3)	total: 181ms	remaining: 8.85s
4:	learn: 3.1596381	test: 3.0282498	best: 3.0282498 (4)	total: 225ms	remaining: 8.77s
5:	learn: 3.1593467	test: 3.0280672	best: 3.0280672 (5)	total: 275ms	remaining: 8.89s
6:	learn: 3.1576406	test: 3.0264308	best: 3.0264308 (6)	total: 320ms	remaining: 8.82s
7:	learn: 3.1569716	test: 3.0257098	best: 3.0257098 (7)	total: 365ms	remaining: 8.75s
8:	learn: 3.1564456	test: 3.0250435	best: 3.0250435 (8)	total: 411ms	remaining: 8.71s
9:	learn: 3.1556515	test: 3.0245645	best: 3.0245645 (9)	total: 453ms	remaining: 8.61s
10:	learn: 3.1555174	test: 3.0245017	best: 3.0245017 (10)	total: 505ms	remaining: 8.67s
11:	learn: 3.1551693	test: 3.0241794	best: 3.02417

[I 2024-03-05 10:36:22,310] Trial 9 finished with value: 0.43523681634184114 and parameters: {'iterations': 200, 'od_wait': 556, 'learning_rate': 0.6364104148996763, 'reg_lambda': 31.435604964072855, 'subsample': 0.7051424146988217, 'random_strength': 46.30265895704372, 'depth': 4, 'min_data_in_leaf': 13, 'leaf_estimation_iterations': 12, 'bagging_temperature': 0.08226074946221039, 'colsample_bylevel': 0.44618794589727584}. Best is trial 8 with value: 0.434446291562222.


139:	learn: 3.0949609	test: 3.0179592	best: 3.0125837 (39)	total: 6.53s	remaining: 2.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 3.012583702
bestIteration = 39

Shrink model to first 40 iterations.
Best Score: 0.434446291562222
Best trial: {'iterations': 900, 'od_wait': 1350, 'learning_rate': 0.11959425474235923, 'reg_lambda': 71.32448158985163, 'subsample': 0.8564710291701385, 'random_strength': 32.45108790277985, 'depth': 12, 'min_data_in_leaf': 15, 'leaf_estimation_iterations': 8, 'bagging_temperature': 0.5130551760589835, 'colsample_bylevel': 0.4152514760464571}


In [ ]:
joblib.dump(best_cat_model, 'best_cat_model.pkl')

['best_cat_model.pkl']

모델 앙상블 가중치 찾기

In [ ]:
def objective(trial):
    # 가중치 탐색 공간 정의
    w1 = trial.suggest_float('w1', 0.0, 1.0)
    w2 = trial.suggest_float('w2', 0.0, 1.0)
    w3 = trial.suggest_float('w3', 0.0, 1.0)

    # 모델 예측 값의 가중 평균
    y_pred = w1 * best_xgb_model.predict(X_test) + w2 * best_lgbm_model.predict(X_test) + w3 * best_cat_model.predict(X_test)

    loss=msle(y_test,y_pred)

    return np.sqrt(loss)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터 값 출력
best_weights = study.best_params
print("Best weights:", best_weights)

[I 2024-03-05 10:37:55,760] A new study created in memory with name: no-name-520975ea-aff4-450f-9834-6ee4314838aa
[I 2024-03-05 10:38:03,018] Trial 0 finished with value: 0.4329194022539829 and parameters: {'w1': 0.14030568456162218, 'w2': 0.46278347662693975, 'w3': 0.3909071905222089}. Best is trial 0 with value: 0.4329194022539829.
[I 2024-03-05 10:38:13,715] Trial 1 finished with value: 1.088574259564003 and parameters: {'w1': 0.9720897709355923, 'w2': 0.8983163463294431, 'w3': 0.9124452099756739}. Best is trial 0 with value: 0.4329194022539829.
[I 2024-03-05 10:38:26,425] Trial 2 finished with value: 0.4276379965942188 and parameters: {'w1': 0.13801909861147965, 'w2': 0.1502310321032474, 'w3': 0.6721428842482066}. Best is trial 2 with value: 0.4276379965942188.
[I 2024-03-05 10:38:40,271] Trial 3 finished with value: 0.7012465224921481 and parameters: {'w1': 0.5324423250679582, 'w2': 0.1923402986770134, 'w3': 0.9837127544959556}. Best is trial 2 with value: 0.4276379965942188.
[I 2

Best weights: {'w1': 0.39036848641135713, 'w2': 0.06958816243523377, 'w3': 0.4188592624169297}


검증set 성능

In [ ]:
from sklearn.metrics import mean_squared_log_error
def rmsle(y, pred):
    return np.sqrt(mean_squared_log_error(y, pred))

In [ ]:
y_pred_valid=best_weights['w1'] * best_xgb_model.predict(X_test) + best_weights['w2'] * best_lgbm_model.predict(X_test) + best_weights['w3'] * best_cat_model.predict(X_test)
rmsle1 = rmsle(y_test, y_pred_valid)
rmsle1

0.42203682543926485

가중치를 이용해 예측값 계산

In [ ]:
y_pred = best_weights['w1'] * best_xgb_model.predict(test_data) + best_weights['w2'] * best_lgbm_model.predict(test_data) + best_weights['w3'] * best_cat_model.predict(test_data)

In [ ]:
y_pred

array([3.94134715, 3.47567516, 4.84201469, ..., 4.3383252 , 4.46241723,
       4.56638654])

파일제출

In [ ]:
sample_submission=pd.read_csv('/content/drive/MyDrive/camp/project1/open/sample_submission.csv')
submit=sample_submission.copy()

In [ ]:
submit['ECLO']=y_pred

In [ ]:
submit

,ID,ECLO
0,ACCIDENT_39609,3.941347
1,ACCIDENT_39610,3.475675
2,ACCIDENT_39611,4.842015
3,ACCIDENT_39612,4.513152
4,ACCIDENT_39613,4.498988
...,...,...
10958,ACCIDENT_50567,5.714144
10959,ACCIDENT_50568,4.260728
10960,ACCIDENT_50569,4.338325
10961,ACCIDENT_50570,4.462417


In [ ]:
submit.to_csv('submision.csv', index=False)



---

##아래부터는 실험한 것

#2

In [ ]:
def objective2(trial):
    # 가중치 탐색 공간 정의
    w1 = trial.suggest_uniform('w1', 0.0, 1.0)
    w2 = trial.suggest_uniform('w2', 0.0, 1.0 - w1)  # w2의 범위를 0부터 1 - w1까지로 설정
    w3 = 1.0 - w1 - w2  # w3는 나머지 값으로 설정


    # 모델 예측 값의 가중 평균
    y_pred = w1 * best_xgb_model.predict(X_test) + w2 * best_lgbm_model.predict(X_test) + w3 * best_cat_model.predict(X_test)

    loss = msle(y_test, y_pred)

    return np.sqrt(loss)

study = optuna.create_study(direction='minimize')
study.optimize(objective2, n_trials=200)

# 최적의 하이퍼파라미터 값 출력
best_weights = study.best_params
print("Best weights:", best_weights)

[I 2024-03-05 03:13:09,646] A new study created in memory with name: no-name-bebc03fa-3924-4bd9-876a-4eae65cbc130
<ipython-input-86-975925b82bfa>:3: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

<ipython-input-86-975925b82bfa>:4: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2024-03-05 03:13:15,265] Trial 0 finished with value: 0.43416010570822544 and parameters: {'w1': 0.5002529849736939, 'w2': 0.4939493890731428}. Best is trial 0 with value: 0.43416010570822544.
<ipython-input-86-975925b82bfa>:3: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

<ipython-input-86-

Best weights: {'w1': 0.23545970363543306, 'w2': 0.41647186240520034}


In [ ]:
best_weights

{'w1': 0.23545970363543306, 'w2': 0.41647186240520034}

In [ ]:
best_weights['w1']+best_weights['w2']+(1-best_weights['w1']-best_weights['w2'])

1.0

In [ ]:
y_pred2 = best_weights['w1'] * best_xgb_model.predict(test_data) + best_weights['w2'] * best_lgbm_model.predict(test_data) + (1-best_weights['w1']-best_weights['w2']) * best_cat_model.predict(test_data)

In [ ]:
submit2=sample_submission.copy()
submit2['ECLO']=y_pred2
submit2.to_csv('submision2.csv', index=False)

In [ ]:
submit2

,ID,ECLO
0,ACCIDENT_39609,4.456222
1,ACCIDENT_39610,3.994828
2,ACCIDENT_39611,5.536461
3,ACCIDENT_39612,5.183055
4,ACCIDENT_39613,5.139735
...,...,...
10958,ACCIDENT_50567,6.559070
10959,ACCIDENT_50568,4.876787
10960,ACCIDENT_50569,4.949647
10961,ACCIDENT_50570,5.063667


#XGBRegressor

In [ ]:
X

,년도,월,시간,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,...,시간구분_새벽,시간구분_심야,시간구분_야간,시간구분_오전,시간구분_오후,시간구분_저녁,계절_가을,계절_겨울,계절_봄,계절_여름
0,2019,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
1,2019,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
2,2019,1,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
3,2019,1,2,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
4,2019,1,4,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,2021,12,23,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
602769,2021,12,23,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
602770,2021,12,23,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
602771,2021,12,23,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
X_train, X_test, y_train, y_test = train_test_split(X,y1, test_size=0.2, random_state=42)

In [ ]:
def rmsle(y, pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y-log_pred)**2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

In [ ]:
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmsle1 = rmsle(y_test, y_pred)
rmsle1

9.11057813107097

In [ ]:
model2 = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6)
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)
mse = mean_squared_error(y_test, y_pred2)
mse

9.12014837074178

In [ ]:
model3 = XGBRegressor(n_estimators=100, learning_rate=0.01, max_depth=3)
model3.fit(X_train, y_train)
y_pred3 = model3.predict(X_test)
mse = mean_squared_error(y_test, y_pred3)
mse

9.174154926384254

In [ ]:
model4 = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=12)
model4.fit(X_train, y_train)
y_pred4 = model4.predict(X_test)
mse = mean_squared_error(y_test, y_pred4)
mse

9.232127623946614

In [ ]:
model5 = XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=3)
model5.fit(X_train, y_train)
y_pred5 = model5.predict(X_test)
mse = mean_squared_error(y_test, y_pred5)
mse

9.123903741520321

lightgbm

In [ ]:
from lightgbm import LGBMRegressor
model6=LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
model6.fit(X_train, y_train)
y_pred6 = model6.predict(X_test)
mse = mean_squared_error(y_test, y_pred6)
mse

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037121 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2061
[LightGBM] [Info] Number of data points in the train set: 213840, number of used features: 1013
[LightGBM] [Info] Start training from score 4.632730
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

9.058311296499996

In [ ]:
model7=LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=6)
model7.fit(X_train, y_train)
y_pred7 = model7.predict(X_test)
mse = mean_squared_error(y_test, y_pred7)
mse

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037375 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2061
[LightGBM] [Info] Number of data points in the train set: 213840, number of used features: 1013
[LightGBM] [Info] Start training from score 4.632730
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

9.048438634000169

In [ ]:
model8=LGBMRegressor(n_estimators=100, learning_rate=0.1,num_leaves=36,  max_depth=6)
model8.fit(X_train, y_train)
y_pred8 = model8.predict(X_test)
mse = mean_squared_error(y_test, y_pred8)
mse

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.056311 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2061
[LightGBM] [Info] Number of data points in the train set: 213840, number of used features: 1013
[LightGBM] [Info] Start training from score 4.632730
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

9.046041814141699

In [ ]:
prediction=model.predict(test_data)

In [ ]:
sample_submission = pd.read_csv(path+"sample_submission.csv")
sample_submission.head()

,ID,ECLO
0,ACCIDENT_39609,0
1,ACCIDENT_39610,0
2,ACCIDENT_39611,0
3,ACCIDENT_39612,0
4,ACCIDENT_39613,0


In [ ]:
baseline_submission = sample_submission.copy()
baseline_submission['ECLO'] = prediction
baseline_submission

,ID,ECLO
0,ACCIDENT_39609,4.416847
1,ACCIDENT_39610,4.178836
2,ACCIDENT_39611,5.618009
3,ACCIDENT_39612,5.340439
4,ACCIDENT_39613,5.247723
...,...,...
10958,ACCIDENT_50567,6.396749
10959,ACCIDENT_50568,5.022019
10960,ACCIDENT_50569,5.067813
10961,ACCIDENT_50570,5.079935


In [ ]:
baseline_submission.to_csv('baseline_submit.csv', index=False)

randomforest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model9 = RandomForestRegressor(n_estimators=100, random_state=42)
model9.fit(X_train, y_train)
y_pred9 = model9.predict(X_test)
mse = mean_squared_error(y_test, y_pred9)
mse

<ipython-input-107-9ab40479986a>:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

